### OCR

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime

import os

import pytesseract
import PIL.Image
if not hasattr(PIL.Image, 'Resampling'):
    PIL.Image.Resampling = PIL.Image
from PIL import Image
from tqdm import tqdm
import re

In [36]:
# image directory
image_dir = '../images/aculei-images/'

In [37]:
# reading the dataframe
df = pd.read_csv('../datasets/metadata.csv', index_col=0)

We want to fill the missing data which were not available through metadata using ocr techniques

The missing data are:

- some **cameras**
- some **date(time)**
- all **temperatures**

So we have to filter the dataframe and try to fill in the missing data for the camera column and then we'll do the same thing for the temparature column (which is not present yet)


In [38]:
df.head()

,hash,image_name,camera,date_time,date,time,moon
0,00100018fdffffff,TF_ACULEI_8040_DSCF0129.jpg,CAM_1,2021-07-22 23:04:07,2021-07-22,23:04:07,Full Moon
1,000000b8fcfcffff,TF_ACULEI_900_DSCF0756.jpg,CAM_1,2021-05-27 22:01:44,2021-05-27,22:01:44,Full Moon
2,0000001000787f7f,TF_ACULEI_15294_DSCF0133.jpg,NaN,2023-03-30 20:35:58,2023-03-30,20:35:58,First Quarter
3,0000001e1e3cfe7f,TF_ACULEI_11374_DSCF0064.jpg,CAM_6,2022-06-09 21:29:59,2022-06-09,21:29:59,Waxing Gibbous
4,00080818fcfeffff,TF_ACULEI_4106_DSCF4336.jpg,CAM_1,2021-06-16 23:41:09,2021-06-16,23:41:09,First Quarter


In [39]:
df.shape

(16874, 7)

In [40]:
df.isnull().sum()

hash             0
image_name       0
camera        4452
date_time      103
date           103
time           103
moon           103
dtype: int64

From the output above we can see we are missing

- 4452 cameras
- 103 dates
- 16874 temperatures

We'll split the jobs to keep it simple

### Cameras

We'll filter the dataset keeping only the empty camera rows (we expect 4452 rows)

In [41]:
df_ocr = df.copy()

In [42]:
df_ocr.shape

(16874, 7)

In [43]:
df_ocr.sample(5)

,hash,image_name,camera,date_time,date,time,moon
7128,00000030f8fcffff,TF_ACULEI_418_DSCF0240.jpg,CAM_1,2021-05-25 02:39:58,2021-05-25,02:39:58,Full Moon
4812,00080878fcfefefe,TF_ACULEI_3894_DSCF4112.jpg,CAM_1,2021-06-16 00:21:45,2021-06-16,00:21:45,First Quarter
910,1e1e3e3e3e1c0c08,TF_ACULEI_8998_DSCF0046.jpg,CAM_2,2021-08-07 23:49:38,2021-08-07,23:49:38,New Moon
16864,0008183c3e3e3e3e,TF_ACULEI_13476_DSCF0247.jpg,NaN,2023-01-16 20:18:02,2023-01-16,20:18:02,Waning Crescent
784,000000001038387e,TF_ACULEI_9499_IMAG0158.jpg,CAM_3,2021-10-11 05:01:47,2021-10-11,05:01:47,Waxing Crescent


Using [pytesseract](https://pypi.org/project/pytesseract/) we can extract text from the images. We decided to crop the images to achieve better results.
The output string from pytesseract is not tokenized or anything so we used RegExp to extract relevant information from the whole string


We had to provide different time formats translations because the metadata about date are diffent from image to image

In [44]:
def parse_date(date_string):
    for format in ["%Y/%m/%d", "%d/%m/%Y"]:
        try:
            return datetime.strptime(date_string, format)
        except ValueError:
            pass
    return None

In [45]:
for index, row in tqdm(df_ocr.iterrows(), total=len(df_ocr)):
    image_path = os.path.join(image_dir, row['image_name'])
    if os.path.isdir(image_path) and image_path.startswith("."):
        continue 
    
    image = Image.open(image_path)
    
    # get image dimensions
    width, height = image.size
    
    # define the coordinates for cropping
    left = 0
    upper = 0
    right = width
    lower = int(height * 17 / 18)  # keep only the bottom 1/3 of the image
    
    # crop the image
    cropped_image = image.crop((left, lower, right, height))
    
    text = pytesseract.image_to_string(cropped_image, config='--psm 3')

    '''
    now let's save the results only if the given column is empty
    save all temperatures 16874
    save some date times 103
    save some cameras 4452
    '''
    
    # saving the camera conditionally
    if pd.isnull(row["camera"]):
        cam_pattern = r'\bCA\S*'
        cam_matches = re.findall(cam_pattern, text)
        row["camera"] = cam_matches[0] if cam_matches else None

    # saving the datetime conditionally
    if pd.isnull(row["date_time"]):
        date_pattern = r'\b\d{4}[-/]\d{2}[-/]\d{2}\b|\b\d{2}[-/]\d{2}[-/]\d{4}\b'    
        time_pattern = r'\b\d{1,2}:\d{2}:\d{2}\b'
        date_matches = re.findall(date_pattern, text)
        time_matches = re.findall(time_pattern, text)

        # preprocessing the date
        if date_matches:
            parsed_date = parse_date(date_matches[0])
            if parsed_date is not None:
                formatted_date = parsed_date.strftime("%Y-%m-%d")
                row["date"] = formatted_date
        
        row["time"] = time_matches[0] if time_matches else None
   
        if time_matches and formatted_date:
            row["date_time"] = formatted_date + " " + time_matches[0] 
    
    # saving the temp always
    celsius_pattern = r'\b\d+\s*[°]\s*C\b'
    celsius_matches = re.findall(celsius_pattern, text)
    df_ocr.loc[index, 'temp'] = celsius_matches[0] if celsius_matches else None

100%|██████████| 16874/16874 [1:49:09<00:00,  2.58it/s]  


Once the process is finished let's save the datasaet and process it in another notebook in order to manage the results safely 

In [47]:
csv_path = '../datasets/ocr.csv'
df_ocr.to_csv(csv_path)